In [ ]:
import pandas as pd
import numpy as np

In [ ]:
original_gas_price_dt = pd.read_csv(r'..\\..\\DataSet\\2004-2021.tsv', sep='\t')
is_gas = original_gas_price_dt['PRODUTO'] == "GASOLINA COMUM"
df_gas = original_gas_price_dt[is_gas]

is_reg = df_gas['REGIÃO'] == "SUDESTE"
df_gas = df_gas[is_reg]

is_est = df_gas['ESTADO'] == "SAO PAULO"
df_gas = df_gas[is_est]

df_gas.head()

In [ ]:
df_gas = df_gas.drop(['PREÇO MÉDIO DISTRIBUIÇÃO',
                          'DESVIO PADRÃO DISTRIBUIÇÃO',
                          'PREÇO MÍNIMO DISTRIBUIÇÃO',
                          'PREÇO MÁXIMO DISTRIBUIÇÃO',
                          'COEF DE VARIAÇÃO DISTRIBUIÇÃO',
                          'DATA FINAL',
                          'PRODUTO',
                          'ESTADO',
                          'DESVIO PADRÃO REVENDA',
                          'PREÇO MÍNIMO REVENDA',
                          'PREÇO MÁXIMO REVENDA',
                          'COEF DE VARIAÇÃO REVENDA',
                          'REGIÃO',
                          'NÚMERO DE POSTOS PESQUISADOS',
                          'UNIDADE DE MEDIDA',
                          'MARGEM MÉDIA REVENDA'], axis=1)

df_gas['DATA INICIAL'] = pd.to_datetime(df_gas['DATA INICIAL'], format='%Y-%m-%d')
df_mask = df_gas['DATA INICIAL'] > '2016-01-09'
df_gas = df_gas[df_mask]
df_gas = df_gas.groupby(pd.Grouper(key='DATA INICIAL', axis=0, freq='W')).mean()

df_gas                

In [ ]:
df_gas.info()

In [ ]:
df_orig_dolar = pd.read_csv(r'..\\Cotação do Dólar por período.csv')
df_orig_dolar = df_orig_dolar.drop(['cotacaoCompra'], axis=1)
df_orig_dolar['dataHoraCotacao'] = pd.to_datetime(df_orig_dolar['dataHoraCotacao'], format='%Y-%m-%d %H:%M:%S')

df_orig_dolar['cotacaoVenda'] = df_orig_dolar['cotacaoVenda'].str.replace(',','.').astype(float)

df_orig_dolar.head()

In [ ]:

# df_orig_dolar['dataHoraCotacao'] = df_orig_dolar['dataHoraCotacao'].apply(lambda x: x.strftime('%Y-%m'))
# df_orig_dolar['dataHoraCotacao'] = pd.to_datetime(df_orig_dolar['dataHoraCotacao'], format='%Y-%m')
df_orig_dolar

In [ ]:
df_orig_dolar = df_orig_dolar.groupby(pd.Grouper(key='dataHoraCotacao', axis=0, freq='W')).mean()
df_orig_dolar

In [ ]:
df_final = df_gas.copy() 
df_final['cotacaoVenda'] = df_orig_dolar['cotacaoVenda']
df_final.rename(columns={
        'PREÇO MÉDIO REVENDA': 'preco_gasolina',
        'cotacaoVenda': 'preco_dolar'

    }, inplace=True)
df_final = df_final.dropna()
#df_final.reset_index(inplace=True)
df_final.head()

# Grafico

In [ ]:
import matplotlib.pyplot as plt 

In [ ]:


plt.scatter(df_final[["preco_gasolina"]], df_final[["preco_dolar"]])
plt.xlabel("Gasolina(R$)")
plt.ylabel("Dolar(R$)")
plt.title("Relação entre preço gasolina e preço dolar")



# Pré-processamento

In [ ]:
#Blibliotecas especificas para o aprendizado de máquina

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import SGDRegressor
from sklearn.neural_network import MLPRegressor



In [ ]:
X = df_final[["preco_dolar"]]
Y = df_final[["preco_gasolina"]].values

In [ ]:
#Normalização
escala = StandardScaler() 
escala.fit(X)

X_norm = escala.transform(X)
# X_norm = X
X_norm

In [ ]:
#Dividir  em conjunto de treinamento e teste 
X_norm_train, X_norm_test, Y_train, Y_test = train_test_split(X_norm, Y, test_size= 0.3)

In [ ]:
# rna = MLPRegressor(hidden_layer_sizes= (50, 25, 10),
#                    max_iter = 2000,
#                    tol = 0.000000001,
#                    learning_rate_init = 0.01,
#                    solver = "sgd",
#                    activation = "relu",
#                    learning_rate = "constant",
#                    verbose = 2,)
rna = MLPRegressor(hidden_layer_sizes= (50, 25, 10),
                   max_iter = 2000,
                   tol = 0.0000001,
                   learning_rate_init = 0.1,
                   solver = "sgd",
                   activation = "logistic",
                   learning_rate = "constant",
                   verbose = 2,)
rna.fit(X_norm_train, Y_train)                   

In [ ]:
reglinear = SGDRegressor(max_iter=2000,
                          tol = 0.0000001,
                         eta0= 0.1,
                         learning_rate="constant",
                         verbose=2,
                        )
reglinear.fit(X_norm_train, Y_train)                    

# Pos- processamento

In [ ]:
#Previssão do conjunto de teste
Y_rna_previsao = rna.predict(X_norm_test)
Y_rl_previsao = reglinear.predict(X_norm_test)

In [ ]:
#Calcular o R^2
r2_rna = r2_score(Y_test,Y_rna_previsao)
r2_rl = r2_score(Y_test,Y_rl_previsao)

print("R2 RNA:", r2_rna)
print("R2 RL:", r2_rl)

In [ ]:
X_test = escala.inverse_transform(X_norm_test)
plt.scatter(X_test, Y_test, alpha = 0.5, label="Reais")
plt.scatter(X_test, Y_rna_previsao, alpha = 0.5, label="MLP")
plt.scatter(X_test, Y_rl_previsao, alpha = 0.5, label="Reg linear")
plt.xlabel("Peso(em kg)")
plt.ylabel("Autonomia (em km/L )")
plt.title ("Comparação dos algoritmos previsto")
plt.legend(loc=1)

# Previsão de Preço

In [ ]:
df_final.info()

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
rates = {}
models = {}
#models['produto'] = reglinear
models['produto'] = rna
rates['2019-01-01'] = df_final['preco_dolar'].iloc[-1]
for i in range(2020, 2025):
    last_year = str(i - 1)+'-01-01'
    curr_year = str(i)+'-01-01'
    rates[curr_year]  = (rates[last_year] * 0.04) + rates[last_year]

rates_df = pd.DataFrame(list(rates.items()), columns = ['DATA', 'preco_dolar'])

rates_df['DATA'] = pd.to_datetime(rates_df['DATA'])
rates_df.set_index('DATA', inplace=True)
predictions = {}

rates_df['PREVISÃO'] = models['produto'].predict(rates_df)
rates_df